## Importing all the modules

In [ ]:
from qiskit_nature.settings import settings #General settings
from qiskit_nature.drivers import UnitsType #Units
from qiskit_nature.drivers.second_quantization import PySCFDriver #Driver = PySCF
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem #Problem = Electronic Structure

settings.dict_aux_operators = True

## Definition of the driver and the problem

In [ ]:
driver = PySCFDriver(atom='O 0.00000 0.00000 0.11779; H 0.00000 0.75545 -0.47116; H 0.00000 -0.75545 -0.47116',
                     unit=UnitsType.ANGSTROM,
                     basis='sto3g')
problem = ElectronicStructureProblem(driver)


Once defined the driver and the problem we can extract the fermionic operators

In [ ]:
## The driver computes the SCF in the molecule and give back all tis properties
properties = problem.driver.run()
print(properties)

Now, if we want the second quantization operators wich can be transformed into Qbits, we can just call any property we want

In [ ]:
second_q_ops = properties.second_q_ops()
print(second_q_ops)

All the information is stored into a dictionary so, if we want the Hamiltonian in the Fock space, we just need the electronic energies times the fock vectors

In [ ]:
hamiltonian = second_q_ops['ElectronicEnergy']
print(hamiltonian)

The Hamiltonian has 1085 terms and the fock vectors has 14 spin orbitals, meaning that if we map 1 to 1 we will need 14 Qbits to define the Hamiltonian.
The numbers in the vector denotes the spin orbital (numbered from 0 to 13) and the + means a creation operator and the - an anihilation operator

## Inspecting the Ground state

To do that we can decide if we want to use all the 14 spin orbitals (full CI-like) or we prefer to constrain the active space (CASSCF like)

### 1. All the 14 spin orbitals

Transformation from fermions to spins (Qbits).
We need to use a Mapper, let's try with Jordan-Wigner with has a 1 to 1 correspondence between occupation number and spin

In [ ]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

jw_mapper = JordanWignerMapper()
jw_converter = QubitConverter(jw_mapper)

In [ ]:
qbit_op_jw = jw_converter.convert(hamiltonian)
print(qbit_op_jw)

The nomenclature of the Qbits is: weight * Pauli term

We can reduce the size of the problem considering symmetries in the Hilbert space of our system. For that we can use a **Parity Mapper** that can remove 2 qbits by exploiting the particle-conserving properties of the electronic structure problems:

In [ ]:
from qiskit_nature.mappers.second_quantization import ParityMapper

parity_mapper = ParityMapper()
parity_converter = QubitConverter(parity_mapper, two_qubit_reduction=True)
## This converter will need as a second argument the number of particles, we can get that from the problem or the properties
qbit_op_parity = parity_converter.convert(hamiltonian,properties.get_property('num_particles'))
## Or directly from problem
qbit_op_parity = parity_converter.convert(hamiltonian,problem.num_particles) ## This solution requires re-run the driver
## in any case:
print(qbit_op_parity)
## This new qbit hamiltonian has 12 qbits instead of 14

Now, to get the ground state energy we will use the variational principle in a quantum algorithm.
Which means the variational equation will be classically evaluated but the expectation values will be calculated in the quantum computer side as the expectation value of the Pauli operator

In [ ]:
## Factory settings
from qiskit.algorithms.optimizers import SLSQP # Optimizer
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator ## Quantum computer simulator
## We need now to build an ansatz for the solver. In this case we will solve the ground state finding the minimum eigenvalues 
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import VQEUCCFactory ## Uses the variational quantum eigensolver with a Unitary Coupled Cluster ansatz

vqe_factory = VQEUCCFactory(quantum_instance=StatevectorSimulator() ,optimizer=SLSQP()) ## if we use instead the Qasm simulator we can analyze the noise

In [ ]:
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
solver_full = GroundStateEigensolver(jw_converter, vqe_factory)
solver_parity = GroundStateEigensolver(parity_converter,vqe_factory)
result_full = solver_full.solve(problem)
result_parity = solver_parity(problem)
## This calculation will take a while with the simulator

In [ ]:
#Print the results
print(result_full)
print(result_parity)

### 2. A Smaller active space
Let's pick an active space with 6 spin orbitals (2 electrons and 3 orbitals)

Qiskit nature has a active space selector implemented

In [ ]:
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer
transformer = ActiveSpaceTransformer(num_electrons=2,num_molecular_orbitals=3)
problem_active_space = ElectronicStructureProblem(driver,[transformer])
second_q_ops_cas = problem_active_space.second_q_ops()
hamiltonian_cas = second_q_ops_cas['ElectronicEnergy']
print(hamiltonian_cas)

Now, that we have a very reduced system we can still apply the parity to get rid of two Qbits

In [ ]:
qbit_op_parity_cas = parity_converter.convert(hamiltonian_cas,problem_active_space.num_particles)
print(qbit_op_parity_cas)
## This new setting uses only 4 Qbits

Now we can get the ground state energy using the same factory as above

In [ ]:
solver_cas = GroundStateEigensolver(parity_converter,vqe_factory)
result_cas = solver_cas.solve(problem_active_space)
print(result_cas)

Doing our own ansatz and sending the calculation to a real quantum computer (simulator)

In [ ]:
from qiskit.circuit.library import EfficientSU2 
ansatz = EfficientSU2(qbit_op_parity_cas.num_qubits, reps=3, entanglement='linear', insert_barriers=True)
#ansatz.decompose().draw('latex', style='iqx')

In [ ]:
from qiskit import Aer # Backend for the simulation

backend = Aer.get_backend('aer_simulator_statevector')


Solving the problem using the Variational Quantum Eigensolver

In [ ]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms import VQE

optimizer = SLSQP()

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend)

result = algorithm.compute_minimum_eigenvalue(qbit_op_parity_cas)
print(result.eigenvalue.real)

electronic_structure_result = problem.interpret(result)
print(electronic_structure_result)

In [ ]:
from qiskit_nature.circuit.library import UCCSD
from qiskit_nature.circuit.library import HartreeFock
init_state = HartreeFock(problem_active_space.num_spin_orbitals,problem_active_space.num_particles, parity_converter)
ansatz = UCCSD(parity_converter,problem_active_space.num_particles, problem_active_space.num_spin_orbitals,reps=3,initial_state=init_state)

In [ ]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms import VQE

optimizer = SLSQP()

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend)

result = algorithm.compute_minimum_eigenvalue(qbit_op_parity_cas)
print(result.eigenvalue.real)

electronic_structure_result = problem.interpret(result)
print(electronic_structure_result)